In [4]:
# modify these for your own computer
repo_directory = '/Users/iaincarmichael/Dropbox/Research/law/law-net/'

data_dir = '/Users/iaincarmichael/Documents/courtlistener/data/'

In [15]:
import sys

# graph package
import igraph as ig

# our code
sys.path.append(repo_directory + 'code/')
from setup_data_dir import setup_data_dir, make_subnetwork_directory
from pipeline.download_data import download_bulk_resource, download_master_edgelist, download_scdb
from helpful_functions import case_info

sys.path.append(repo_directory + 'explore/vertex_metrics_experiment/code/')
from make_network_data import *
from bag_of_words import make_tf_idf
from experiment_helper_functions import *

# which network
network_name = 'scotus'

# some sub directories that get used
raw_dir = data_dir + 'raw/'
subnet_dir = data_dir + network_name + '/'
text_dir = subnet_dir + 'textfiles/'


# jupyter notebook settings
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
G = ig.Graph.Read_GraphML(subnet_dir + network_name +'_network.graphml')

In [11]:
test_params = {'active_years':  range(1900, 2015 + 1),
               'seed': 4332,
               'num_test_cases': 10}

In [12]:
test_cases = get_test_cases(G,
                            test_params['active_years'],
                            test_params['num_test_cases'],
                            test_params['seed'])

In [14]:
tfidf_matrix, op_id_to_bow_id = load_tf_idf(subnet_dir + 'nlp/')

In [25]:
tc = 0
v = test_cases[tc]

snapshot_year = test_case['year'] - 1

ancentors = [v.index for v in G.vs.select(year_le=snapshot_year)]

# all edges from ing case to previous cases
edgelist = zip([test_case.index] * len(ancentors), ancentors)


ed_CLids = [G.vs[edge[1]]['name'] for edge in edgelist]
ing_CLids = [G.vs[edge[0]]['name'] for edge in edgelist]

In [26]:
%time S = compute_similarities(tfidf_matrix, zip(ing_CLids, ed_CLids), op_id_to_bow_id)

CPU times: user 20.9 s, sys: 349 ms, total: 21.3 s
Wall time: 28.5 s
